In [1]:
!pip install keras
!pip install tensorflow

In [2]:
import nltk
import json
import pickle
import numpy as np
import random
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD


text=[]
dataclass = []
doc = []
avoid = ['?', '!']
file = open('data.json').read()
res = json.loads(file)

In [3]:
import nltk
nltk.download('punkt')

for ints in res['intents']:
    
    for pattern in ints['patterns']:

        #tokenize each word
        f = nltk.word_tokenize(pattern)
        text.extend(f)
        
        doc.append((f, ints['tag']))

        if ints['tag'] not in dataclass:
            
            dataclass.append(ints['tag'])

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
import nltk
nltk.download('wordnet')

# lowercase the word and lemmatize 
text = [lemmatizer.lemmatize(x.lower()) for x in text if x not in avoid]

text = sorted(list(set(text)))

dataclass = sorted(list(set(dataclass)))

print (len(doc), "documents")

print (len(dataclass), "classes", dataclass)

print (len(text), "words", text)


pickle.dump(text,open('text.pkl','wb'))

pickle.dump(dataclass,open('class.pkl','wb'))

[nltk_data] Downloading package wordnet to /home/ec2-user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


114 documents
26 classes ['ATM', 'Account doubts', 'Account number doubts', 'Business', 'Card', 'Current', 'Monthly Statement', 'Statement', 'accountbalance', 'credit card', 'deposit doubts', 'faq', 'goodbye', 'greeting', 'my card expiring', 'open a account', 'pay and transfer', 'profile faq', 'savings', 'schedule transfer', 'six month Statement', 'thanks', 'transfer limit', 'update contact info', 'username doubts', 'yearly Statement']
99 words ["'s", ',', '/', '1', '6', 'a', 'account', 'all', 'and', 'anyone', 'are', 'asked', 'atm', 'awesome', 'balance', 'branch', 'business', 'bye', 'card', 'change', 'chatting', 'check', 'contact', 'credit', 'current', 'day', 'deposit', 'detail', 'do', 'doubt', 'expired', 'expiring', 'faq', 'for', 'frequently', 'good', 'goodbye', 'half', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'info', 'information', 'is', 'later', 'limit', 'location', 'me', 'month', 'monthly', 'my', 'near', 'new', 'next', 'nice', 'number', 'open', 'openi

In [5]:
train = []

empty = [0] * len(dataclass)

for d in doc:

    bow = []
    pattern = d[0]
    
    pattern = [lemmatizer.lemmatize(w.lower()) for w in pattern]

    for w in text:
        bow.append(1) if w in pattern else bow.append(0)
    
    row = list(empty)
    row[dataclass.index(d[1])] = 1
    
    train.append([bow, row])

random.shuffle(train)
train = np.array(train)

training1 = list(train[:,0])
training2 = list(train[:,1])

print("Created train data")

Created train data


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [6]:
model = Sequential()
model.add(Dense(128, input_shape=(len(training1[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(training2[0]), activation='softmax'))

gradient = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=gradient, metrics=['accuracy'])

fitting = model.fit(np.array(training1), np.array(training2), epochs=220, batch_size=10, verbose=1)
model.save('trained_model.h5', fitting)

print("Created")


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/220
12/12 [==============================] - 0s 1ms/step - loss: 3.2866 - accuracy: 0.0175
Epoch 2/220
12/12 [==============================] - 0s 1ms/step - loss: 3.2131 - accuracy: 0.0526
Epoch 3/220
12/12 [==============================] - 0s 1ms/step - loss: 3.1619 - accuracy: 0.1053
Epoch 4/220
12/12 [==============================] - 0s 1ms/step - loss: 3.1487 - accuracy: 0.1053
Epoch 5/220
12/12 [==============================] - 0s 1ms/step - loss: 3.0931 - accuracy: 0.1667
Epoch 6/220
12/12 [==============================] - 0s 1ms/step - loss: 3.0329 - accuracy: 0.1228
Epoch 7/220
12/12 [==============================] - 0s 1ms/step - loss: 2.9766 - accuracy: 0.1754
Epoch 8/220
12/12 [==============================] - 0s 1ms/step - loss: 2.9581 - accuracy: 0.2105
Epoch 9/220
12/12 [==============================] - 0s 1ms/step - loss: 2.8662 - accuracy: 0.2105
Epoch 10/220
12/12 [==============================] - 0s 1ms/step - loss: 2.7763 - accuracy: 0.2368
Epoch 11/